In [2]:
import sys
import numpy as np
import pandas as pd
sys.path.append('../')
from utils import *
from bayes_ensemble.bayes_ensemble import *
import warnings
warnings.filterwarnings("ignore")

In [12]:
df_preds_all, models_by_state, data_all = load_preds(rename = False)

df_preds_all = df_preds_all.loc[df_preds_all.model_id.isin(['21', '22', '25', '26', '27', '28', '30', '34'] )]

df_preds_all.head()

,date,pred,lower,upper,state,model_id
0,2022-10-09,110.690113,64.291013,219.826948,AC,21
2800,2025-08-10,192.641953,142.814506,261.828157,PR,21
2801,2025-08-17,171.137457,119.943438,233.918928,PR,21
2802,2025-08-24,196.875395,132.696765,267.461459,PR,21
2803,2025-08-31,179.610112,136.303073,247.230651,PR,21


In [13]:
state = 'SP'

In [14]:
data_23, data_24, preds_23, preds_24, preds_25, models = format_data(state, models_by_state, data_all, df_preds_all)

K = len(models)
weights = np.full(K, 1/(K))

ens_24 = Ensemble_linear(df= preds_24,
            order_models = models)

df_24_out = ens_24.apply_ensemble(weights = weights)

df_24_out.head()

,pred,lower,upper,date
0,7097.934860,2356.416746,24803.457014,2024-05-26
1,4812.710632,1701.813354,15623.882776,2024-06-02
2,9799.289895,3126.530315,32968.854448,2024-05-19
3,14744.219437,4176.310992,43650.152486,2024-05-12
4,19207.505289,5951.918691,57916.058858,2024-04-28


df_24_out.to_csv('teste_SP_base_2024.csv.gz')

In [27]:
def compute_ppf(mu, sigma, weights):
    """
    Compute the Percent-Point Function (PPF), which is the inverse of the CDF, 
    for a mixture of lognormal distributions.

    The function takes the parameters of a lognormal mixture (mean, standard deviation, and weights) 
    and returns the mixture values for the 5th, 50th, and 95th percentiles.

    Parameters:
        mu (array-like): Mean values (in log-space) for the lognormal components of the mixture.
        sigma (array-like): Standard deviation values (in log-space) for the lognormal components of the mixture.
        weights (array-like): Weights of each component in the lognormal mixture. These should sum to 1.

    Returns:
        np.ndarray: The x-values corresponding to the 5th, 50th, and 95th percentiles. 
    """
    # Step 1: Generate a range of x-values for the integration
    x = np.linspace(1e-6, 10**5, 10**5)
    
    # Step 2: Compute PDF values for the mixture of lognormal distributions
    pdf_values = dlnorm_mix(x, mu, sigma, weights, log=False)

    # Step 3: Normalize the PDF using the trapezoidal rule
    dx = np.diff(x)  # Compute spacing between consecutive x-values
    dx = np.append(dx, dx[-1])  # Ensure length matches the x array
    area = np.sum(pdf_values * dx)  # Approximate the area under the PDF
    pdf_values_normalized = pdf_values / area  # Normalize the PDF to ensure total area is 1

    # Step 4: Compute the CDF by integrating the normalized PDF using the trapezoidal rule
    cdf_values = cumulative_trapezoid(pdf_values_normalized, x, initial=0)

    # Step 5: Invert the CDF to obtain the PPF
    ppf_function = interp1d(cdf_values, x, bounds_error=False, fill_value="extrapolate")
    
    # Example: Given probabilities, find the corresponding x values
    p = [0.5, 0.05, 0.95]  # Example probabilities (e.g., 5th, 50th, and 95th percentiles)
    x_for_p = ppf_function(p)  # Get x-values corresponding to the probabilities

    return x_for_p

def compute_ppf2(mu, sigma, weights):
    """
    Compute the Percent-Point Function (PPF), which is the inverse of the CDF, 
    for a mixture of lognormal distributions.

    The function takes the parameters of a lognormal mixture (mean, standard deviation, and weights) 
    and returns the mixture values for the 5th, 50th, and 95th percentiles.

    Parameters:
        mu (array-like): Mean values (in log-space) for the lognormal components of the mixture.
        sigma (array-like): Standard deviation values (in log-space) for the lognormal components of the mixture.
        weights (array-like): Weights of each component in the lognormal mixture. These should sum to 1.

    Returns:
        np.ndarray: The x-values corresponding to the 5th, 50th, and 95th percentiles. 
    """
    # Step 1: Generate a range of x-values for the integration
    x = np.linspace(1e-6, 10**5, 10**5)
    
    # Step 2: Compute PDF values for the mixture of lognormal distributions
    pdf_values = dlnorm_mix(x, mu, sigma, weights, log=False)

    mu, sigma = lognorm.fit(pdf_values)

    p = [0.5, 0.05, 0.95]  
    x_for_p = lognorm.ppf(p, loc=mu,  scale=sigma)

    return x_for_p
    

In [28]:
compute_ppf([5,5], [10,10], [0.5, 0.5])

array([2.18668288e+02, 1.00008167e-01, 9.47487971e+09])

In [29]:
compute_ppf2([5,5], [10,10], [0.5, 0.5])

ValueError: too many values to unpack (expected 2)